### Import Modules

In [1]:
import os, sys
import skimage as sk
import numpy as np
# import json 
# import cv2
# from imantics import Mask
# import albumentations as A
# import yaml
# import re
import matplotlib.image as mpimg

import matplotlib.pyplot as plt
from pathlib import Path
import random
from convert_npy_to_jpg import extract_raw_data
from annotate import *
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 16
%matplotlib qt

### Load random QFlow image

In [2]:
image_folder = Path("../data/train")
npy_files = list(image_folder.glob("*.npy"))
random_npy_file = random.choice(npy_files)
corresponding_image = image_folder / (random_npy_file.stem + ".jpg")

### Visualize the pre-defined dot configurations

In [3]:
image = mpimg.imread(corresponding_image)
states, V_P1, V_P2 = extract_raw_data(random_npy_file, data_type='state')
plt.figure(layout='tight')
plt.imshow(
    image,
    extent=[V_P1.min(), V_P1.max(), V_P2.min(), V_P2.max()],
    alpha=1,
    cmap='binary',
    origin='lower'
    )
plt.imshow(
    states,
    extent=[V_P1.min(), V_P1.max(), V_P2.min(), V_P2.max()],
    alpha=0.5,
    origin='lower'
    )
plt.ylabel(r"$V_{P_2}$ (V)")
plt.xlabel(r"$V_{P_1}$ (V)")
plt.title("QFlow Dot Configuration States")
cbar = plt.colorbar()
cbar.set_ticks([-1, 0, 0.5, 1, 1.5, 2])
plt.show()

### Convert pre-defined dot configurations to polygons

Helper functions

In [6]:
class_dict = {0.0: "ND", -1.0: "ND", 0.5: "LD", 1.0: "CD", 1.5: "RD", 2.0: "DD"}
color_dict = {'ND': "k", "ND": "k", "LD": "r", "CD": "g", "RD": "b", "DD": "y"}

background_value = -1 # barrier or no dot
correction = 2 # Need integer values for Mask-RCNN

states_regions_labelled = sk.measure.label(
    (correction * states).astype(np.uint8), 
    background=background_value, 
    connectivity=1
)

states_regions = sk.measure.regionprops(states_regions_labelled)
plt.figure(layout='tight')
for index, region in enumerate(states_regions):
    region_coords = region.coords
    
    # Get boundaries of coordinates
    temp = {}
    for row in region_coords:
        key = row[0]
        value = row[1]
        if key not in temp:
            temp[key] = [value, value]  # Initialize with the current value
        else:
            temp[key][0] = min(temp[key][0], value)  # Update minimum value
            temp[key][1] = max(temp[key][1], value)  # Update maximum value

    boundary_coords = np.array([[key, minmax[0]] for key, minmax in temp.items()] + [[key, minmax[1]] for key, minmax in temp.items()])

    y, x = boundary_coords.T
    polygon = np.vstack((x,y)).T

    # Filter any weird artifacts for detectron2
    if len(x) <= 10 or len(y) <= 10:
        continue
    
    polygon_clockwise = organize_array_clockwise(polygon)
    centroid = find_polygon_centroid(polygon_clockwise)
    centroid_y, centroid_x = centroid
    polygon_state = states[
        centroid_x.astype(int), 
        centroid_y.astype(int)
        ]
    polygon_class = class_dict[polygon_state]

    # Patches missing LD, RD info in v2 simulated
    if centroid_x <= 20 and polygon_class == 'CD':
        polygon_class = 'RD'
    if centroid_y <= 20 and polygon_class == 'CD':
        polygon_class = 'LD'
        
    plt.text(*centroid,s=polygon_class)
    plt.scatter(polygon_clockwise[:,0], polygon_clockwise[:,1], c=color_dict[polygon_class])
    plt.scatter(*centroid,c=color_dict[polygon_class])
plt.xlabel("X Index")
plt.ylabel("Y Index")
plt.title("Polygons Extracted")
plt.show()